In [1]:
# imports
import configparser
import os
from bs4 import BeautifulSoup 
import requests 
from time import sleep 
from collections import Counter 
import pandas as pd 
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
import re           
import datetime
import seaborn as sns
from matplotlib import pyplot as plt
import gcloud 
from google.cloud import storage


%matplotlib inline

In [2]:
"""I start by writing a simple function to create a list of url's to scrape, representing each year of F1 races"""

def get_urls():
    global YEARS
    global all_race_url
    global url
    
    YEARS=list(range(1950,2024)) 
    all_race_url=[]
    for year in YEARS:
        url=f'https://www.formula1.com/en/results.html/{year}/races.html'
        all_race_url.append(url)
    return all_race_url

get_urls()

#^^^ We know F1 has been going since 1950, so 'YEARS', is a list of numerical values from 1950-2023 ^^^
#^^^ Next I wrote a simple loop, utilizing f string to iterate through the list of YEARS 
# and input year into the url and appended to an empty list as 'all_race_url'

['https://www.formula1.com/en/results.html/1950/races.html',
 'https://www.formula1.com/en/results.html/1951/races.html',
 'https://www.formula1.com/en/results.html/1952/races.html',
 'https://www.formula1.com/en/results.html/1953/races.html',
 'https://www.formula1.com/en/results.html/1954/races.html',
 'https://www.formula1.com/en/results.html/1955/races.html',
 'https://www.formula1.com/en/results.html/1956/races.html',
 'https://www.formula1.com/en/results.html/1957/races.html',
 'https://www.formula1.com/en/results.html/1958/races.html',
 'https://www.formula1.com/en/results.html/1959/races.html',
 'https://www.formula1.com/en/results.html/1960/races.html',
 'https://www.formula1.com/en/results.html/1961/races.html',
 'https://www.formula1.com/en/results.html/1962/races.html',
 'https://www.formula1.com/en/results.html/1963/races.html',
 'https://www.formula1.com/en/results.html/1964/races.html',
 'https://www.formula1.com/en/results.html/1965/races.html',
 'https://www.formula1.c

In [3]:

"""Function to check status_code for url requests.
    Will iterate through list of url's"""

def status_code():
    global result
    global soup
    for url in all_race_url:
        result=requests.get(url)
        soup=BeautifulSoup(result.content)
        result.status_code
        if result.status_code != 200:
            print(url,'STATUS_CODE ERROR')
            break
        else:
            None
    print('SUCCESS: ALL STATUS CODES=200')
    
status_code()

#^^^ Function to iterate through the list of url's in 'all_race_url' and check status_code for each url, will return ERROR message and url if url status_code is not 200 ^^^

SUCCESS: ALL STATUS CODES=200


In [4]:

"""Function to scrape, clean, create pandas dataframes and then join all into 1 table"""

def get_page_data():
    global gp_list
    global winners_list
    global teams_list
    global lap_list
    global times_list
    global dates_list
    
    gp_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        grand_prix_list=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='dark bold')][0::2]
        gp_list.append(grand_prix_list)
        #^^^ Collect all GP's to a list of lists from all urls^^^
        
    winners_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        race_winner=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='dark bold')][1::2]
        winners_list.append(race_winner)
        #^^^ Collect all race winner's to a list of lists from all urls^^^
        
    teams_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        team_list=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='semi-bold uppercase')]
        teams_list.append(team_list)
        #^^^ Collect all winning teams to a list of lists from all urls^^^
        
    lap_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        laps_list=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='bold hide-for-mobile')]
        lap_list.append(laps_list)
        #^^^ Collect all lap counts to a list of lists from all urls^^^
        
    times_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        time_list=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='dark bold hide-for-tablet')]
        times_list.append(time_list)
        #^^^ Collect all race times to a list of lists from all urls^^^
        
    dates_list=[]
    for x in all_race_url:
        result=requests.get(x)
        soup=BeautifulSoup(result.content, 'lxml')
        race_date_list=[tag.get_text(strip=True) for tag in soup.find_all('td', class_='dark hide-for-mobile')]
        dates_list.append(race_date_list)
        #^^^ Collect all race dates to a list of lists from all urls^^^
        
        #^^^ Collecting all desired columns ^^^
    #return teams_list

get_page_data()

In [5]:

""" Function will append list of lists from previous function (get_page_data) 
to a list of all values for each column """

def one_list():
    global full_gp
    global full_winner
    global full_teams
    global full_lap
    global full_times
    global full_dates


    full_gp=[]
    for l in gp_list:
        for x in l:
            full_gp.append(x)
    #^^^ Append gp data into new list full_gp ^^^
            
    full_winner=[]
    for l in winners_list:
        for x in l:
            x=x[:-3]
            full_winner.append(x)
    #^^^ Append winners data into new list full_winner ^^^
    #^^^ Cleaning up the values in the winners columns by droping last 3 characters ^^^
            
    full_teams=[]
    for l in teams_list:
        for x in l:
            full_teams.append(x)
    #^^^ Append teams data into new list full_winner ^^^
            
    full_lap=[]
    for l in lap_list:
        for x in l:
            full_lap.append(x)
    #^^^ Append lap data into new list full_winner ^^^
            
    full_times=[]
    for l in times_list:
        for x in l:
            x=x[:-4]
            full_times.append(x.split(':'))
    #^^^ Append time data into new list full_winner ^^^
    #^^^ Split up time column by hour, min, sec ^^^
            
    full_dates=[]
    for l in dates_list:
        for x in l:
            full_dates.append(x)    
    #^^^ Append dates data into new list full_winner ^^^
            
    #return full_times

one_list()

In [6]:

""" Create pandas dataframes for each list created in the previous step & concat the lists, column wise """

full_gp=pd.DataFrame(full_gp)
full_winner=pd.DataFrame(full_winner)
full_teams=pd.DataFrame(full_teams)
full_dates=pd.DataFrame(full_dates)
full_lap=pd.DataFrame(full_lap)
full_times=pd.DataFrame(full_times)
#^^^ Creating pandas dataframes for each list of values ^^^
    
full_df=pd.concat([full_dates, full_gp, full_winner, full_teams, full_times,full_lap], axis='columns')
#^^^ Combine dataframes into new dataframe (full_df) ^^^
full_df

,0,0,0,0,0,1,2,0
0,13 May 1950,Great Britain,NinoFarina,Alfa Romeo,2,13,23,70
1,21 May 1950,Monaco,Juan ManuelFangio,Alfa Romeo,3,13,18,100
2,30 May 1950,Indianapolis 500,JohnnieParsons,Kurtis Kraft Offenhauser,2,46,55,138
3,04 Jun 1950,Switzerland,NinoFarina,Alfa Romeo,2,02,53,42
4,18 Jun 1950,Belgium,Juan ManuelFangio,Alfa Romeo,2,47,26,35
...,...,...,...,...,...,...,...,...
1099,22 Oct 2023,United States,MaxVerstappen,Red Bull Racing Honda RBPT,1,35,21,56
1100,29 Oct 2023,Mexico,MaxVerstappen,Red Bull Racing Honda RBPT,2,02,30,71
1101,05 Nov 2023,Brazil,MaxVerstappen,Red Bull Racing Honda RBPT,1,56,48,71
1102,18 Nov 2023,Las Vegas,MaxVerstappen,Red Bull Racing Honda RBPT,1,29,08,50


In [7]:

""" Assign column names to the new dataframe & write dataframe to csv format and store locally """

full_df.columns=['Date', 'Grand_Prix', 'Winner', 'Team', 'finish_time_H', 'finish_time_M', 'finish_time_S', 'Laps']

full_df.to_csv('f1_full_df.csv')
full_df.head(10)

,Date,Grand_Prix,Winner,Team,finish_time_H,finish_time_M,finish_time_S,Laps
0,13 May 1950,Great Britain,NinoFarina,Alfa Romeo,2,13,23,70
1,21 May 1950,Monaco,Juan ManuelFangio,Alfa Romeo,3,13,18,100
2,30 May 1950,Indianapolis 500,JohnnieParsons,Kurtis Kraft Offenhauser,2,46,55,138
3,04 Jun 1950,Switzerland,NinoFarina,Alfa Romeo,2,02,53,42
4,18 Jun 1950,Belgium,Juan ManuelFangio,Alfa Romeo,2,47,26,35
5,02 Jul 1950,France,Juan ManuelFangio,Alfa Romeo,2,57,52,64
6,03 Sep 1950,Italy,NinoFarina,Alfa Romeo,2,51,17,80
7,27 May 1951,Switzerland,Juan ManuelFangio,Alfa Romeo,2,07,53,42
8,30 May 1951,Indianapolis 500,LeeWallard,Kurtis Kraft Offenhauser,3,57,38,200
9,17 Jun 1951,Belgium,NinoFarina,Alfa Romeo,2,45,46,36


In [8]:

""" Check which columns have null values, if any"""

full_df.isnull().any()

Date             False
Grand_Prix       False
Winner           False
Team             False
finish_time_H    False
finish_time_M     True
finish_time_S     True
Laps             False
dtype: bool

In [9]:

""" After finding null values, we will use .replace() and .fillna() 
to replace empty string values with the value 0 and replace nan values with 0 """

full_df['finish_time_H']=full_df['finish_time_H'].replace('', 0)
full_df['finish_time_M']=full_df['finish_time_M'].replace('', 0)
full_df['finish_time_S']=full_df['finish_time_S'].replace('', 0)

full_df['finish_time_H']=full_df['finish_time_H'].fillna(0) 
full_df['finish_time_M']=full_df['finish_time_M'].fillna(0)
full_df['finish_time_S']=full_df['finish_time_S'].fillna(0)

In [10]:

""" Check again which columns have null values, 
to see if the last step took care of these values """

full_df.isnull().any()

Date             False
Grand_Prix       False
Winner           False
Team             False
finish_time_H    False
finish_time_M    False
finish_time_S    False
Laps             False
dtype: bool

In [11]:

""" Next we convert the hour, min, sec columns as INTEGER types and convert values to seconds """

full_df['finish_time_H']=full_df['finish_time_H'].astype(int)*3600
#^^^ We multiply the hour values by 3600 to get seconds conversion ^^^

full_df['finish_time_M']=full_df['finish_time_M'].astype(int)*60
#^^^ We multiply the min values by 60 to get seconds conversion ^^^

full_df['finish_time_S']=full_df['finish_time_S'].astype(int)
full_df

,Date,Grand_Prix,Winner,Team,finish_time_H,finish_time_M,finish_time_S,Laps
0,13 May 1950,Great Britain,NinoFarina,Alfa Romeo,7200,780,23,70
1,21 May 1950,Monaco,Juan ManuelFangio,Alfa Romeo,10800,780,18,100
2,30 May 1950,Indianapolis 500,JohnnieParsons,Kurtis Kraft Offenhauser,7200,2760,55,138
3,04 Jun 1950,Switzerland,NinoFarina,Alfa Romeo,7200,120,53,42
4,18 Jun 1950,Belgium,Juan ManuelFangio,Alfa Romeo,7200,2820,26,35
...,...,...,...,...,...,...,...,...
1099,22 Oct 2023,United States,MaxVerstappen,Red Bull Racing Honda RBPT,3600,2100,21,56
1100,29 Oct 2023,Mexico,MaxVerstappen,Red Bull Racing Honda RBPT,7200,120,30,71
1101,05 Nov 2023,Brazil,MaxVerstappen,Red Bull Racing Honda RBPT,3600,3360,48,71
1102,18 Nov 2023,Las Vegas,MaxVerstappen,Red Bull Racing Honda RBPT,3600,1740,8,50


In [12]:

""" Next we convert the hour, min and seconds columns
to get our total race time in seconds and assign this column the name RaceTime_sec"""

full_df['RaceTime_sec']=(full_df['finish_time_H']+full_df['finish_time_M']+full_df['finish_time_S'])
full_df

,Date,Grand_Prix,Winner,Team,finish_time_H,finish_time_M,finish_time_S,Laps,RaceTime_sec
0,13 May 1950,Great Britain,NinoFarina,Alfa Romeo,7200,780,23,70,8003
1,21 May 1950,Monaco,Juan ManuelFangio,Alfa Romeo,10800,780,18,100,11598
2,30 May 1950,Indianapolis 500,JohnnieParsons,Kurtis Kraft Offenhauser,7200,2760,55,138,10015
3,04 Jun 1950,Switzerland,NinoFarina,Alfa Romeo,7200,120,53,42,7373
4,18 Jun 1950,Belgium,Juan ManuelFangio,Alfa Romeo,7200,2820,26,35,10046
...,...,...,...,...,...,...,...,...,...
1099,22 Oct 2023,United States,MaxVerstappen,Red Bull Racing Honda RBPT,3600,2100,21,56,5721
1100,29 Oct 2023,Mexico,MaxVerstappen,Red Bull Racing Honda RBPT,7200,120,30,71,7350
1101,05 Nov 2023,Brazil,MaxVerstappen,Red Bull Racing Honda RBPT,3600,3360,48,71,7008
1102,18 Nov 2023,Las Vegas,MaxVerstappen,Red Bull Racing Honda RBPT,3600,1740,8,50,5348


In [13]:

""" Next we reassign our full_df variable to a new dataframe and keep only
'Date', 'Grand_Prix', 'Winner', 'Team', 'Laps', 'RaceTime_sec' columns """

full_df=full_df[['Date', 'Grand_Prix', 'Winner', 'Team', 'Laps', 'RaceTime_sec']]


""" Next we replace all nan and empty string values from the Laps column and cast it as a INTEGER """

full_df['Laps'].replace('', 0, inplace=True)
full_df['Laps'].replace('null', 0, inplace=True)
full_df['Laps'].fillna(0, inplace=True)
full_df.isnull().any()
full_df['Laps']=full_df['Laps'].astype(int)

/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/1929127991.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['Laps'].replace('', 0, inplace=True)
/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/1929127991.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['Laps'].replace('null', 0, inplace=True)
/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/1929127991.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [14]:

""" Now I create a new column called AVG_Lap_Time_sec by 
dividing the RaceTime_sec column by number of Laps per GP. 
This will give us the average time it took the winner per lap """

full_df['AVG_Lap_Time_sec']=(full_df['RaceTime_sec']/full_df['Laps'])
full_df

full_df.to_csv('F1_avg_lap_df.csv')

/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/3453314773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['AVG_Lap_Time_sec']=(full_df['RaceTime_sec']/full_df['Laps'])


In [15]:
full_df

,Date,Grand_Prix,Winner,Team,Laps,RaceTime_sec,AVG_Lap_Time_sec
0,13 May 1950,Great Britain,NinoFarina,Alfa Romeo,70,8003,114.328571
1,21 May 1950,Monaco,Juan ManuelFangio,Alfa Romeo,100,11598,115.980000
2,30 May 1950,Indianapolis 500,JohnnieParsons,Kurtis Kraft Offenhauser,138,10015,72.572464
3,04 Jun 1950,Switzerland,NinoFarina,Alfa Romeo,42,7373,175.547619
4,18 Jun 1950,Belgium,Juan ManuelFangio,Alfa Romeo,35,10046,287.028571
...,...,...,...,...,...,...,...
1099,22 Oct 2023,United States,MaxVerstappen,Red Bull Racing Honda RBPT,56,5721,102.160714
1100,29 Oct 2023,Mexico,MaxVerstappen,Red Bull Racing Honda RBPT,71,7350,103.521127
1101,05 Nov 2023,Brazil,MaxVerstappen,Red Bull Racing Honda RBPT,71,7008,98.704225
1102,18 Nov 2023,Las Vegas,MaxVerstappen,Red Bull Racing Honda RBPT,50,5348,106.960000


In [16]:

""" Now we can convert the date column to a pandas datetime type, 
sort the values by date in ascending order and then set index as the date column """

full_df['Date']=pd.to_datetime(full_df['Date'], format='mixed')
full_df.sort_values(by=['Date'], ascending=False, inplace=True)
full_df.set_index('Date', inplace=True)


/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/4091163772.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['Date']=pd.to_datetime(full_df['Date'], format='mixed')
/var/folders/j1/qffd_d6d1x36qx1ml0nk7tbc0000gn/T/ipykernel_68090/4091163772.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.sort_values(by=['Date'], ascending=False, inplace=True)


In [17]:

""" Now we write the full_df file to csv format and save locally """

full_df.to_csv('f1_final_df.csv')

In [18]:
# Function to upload or overwrite csv file to desired GCP Cloud Storage Bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='gcp-creds.json'


def upload_file_to_bucket(bucket_name, source_file_name, destination_blob_name):
    
    """Uploads a file to the GCP bucket."""
    
    client = storage.Client()
    bucket = client.get_bucket('f1_winners_historical')
    bucket_name = "f1_winners_historical"
    source_file_name = "f1_final_df.csv"
    destination_blob_name = "f1_final_df.csv"

    client = storage.Client()
    bucket = client.bucket('f1_winners_historical')
    blob = bucket.blob('f1_final_df.csv')

    blob.upload_from_filename('f1_final_df.csv')

    print(
        "File {} uploaded to {}.".format(
            source_file_name, bucket_name))

In [19]:
upload_file_to_bucket("f1_winners_historical", "f1_final_df.csv", "f1_final_df.csv")


File f1_final_df.csv uploaded to f1_winners_historical.


In [20]:
###############################
###############################
###############################

# API TESTING

In [21]:
from timple.timedelta import strftimedelta

import fastf1
import fastf1.plotting
from fastf1.core import Laps


In [22]:
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None,
                          misc_mpl_mods=False)

session = fastf1.get_session(2023, 'Spanish Grand Prix', 'Q')
session.load()


from fastf1.ergast import Ergast
ergast = Ergast(result_type='pandas', auto_cast=True)


req         WARNING 	DEFAULT CACHE ENABLED! (546.83 MB) /Users/mannybrar/Library/Caches/fastf1
core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '4', '10', '44', '18', '31', '27', '14', '81', '11', '63', '24', '21', '22', '77', '20', '23', '16', '2']


In [23]:
circuit_=ergast.get_circuits()
#circuit_

In [24]:
### RAW  response from quali
quali_=ergast.get_qualifying_results(season=2023, round=3, result_type='raw', limit=None, auto_cast=True)
#quali_[1]

In [25]:
# Flatten JSON to pandas df - Extract nested QualifyingResults

quali_result=pd.json_normalize(quali_, ['QualifyingResults'])

In [26]:
quali_result

,number,position,Q1,Q2,Q3,Driver.driverId,Driver.permanentNumber,Driver.code,Driver.url,Driver.givenName,Driver.familyName,Driver.dateOfBirth,Driver.nationality,Constructor.constructorId,Constructor.url,Constructor.name,Constructor.nationality
0,1,1,1:17.384,1:17.056,1:16.732,max_verstappen,33,VER,http://en.wikipedia.org/wiki/Max_Verstappen,Max,Verstappen,1997-09-30,Dutch,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian
1,63,2,1:17.654,1:17.513,1:16.968,russell,63,RUS,http://en.wikipedia.org/wiki/George_Russell_(r...,George,Russell,1998-02-15,British,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German
2,44,3,1:17.689,1:17.551,1:17.104,hamilton,44,HAM,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis,Hamilton,1985-01-07,British,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German
3,14,4,1:17.832,1:17.283,1:17.139,alonso,14,ALO,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando,Alonso,1981-07-29,Spanish,aston_martin,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,Aston Martin,British
4,55,5,1:17.928,1:17.349,1:17.270,sainz,55,SAI,http://en.wikipedia.org/wiki/Carlos_Sainz_Jr.,Carlos,Sainz,1994-09-01,Spanish,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian
5,18,6,1:17.873,1:17.616,1:17.308,stroll,18,STR,http://en.wikipedia.org/wiki/Lance_Stroll,Lance,Stroll,1998-10-29,Canadian,aston_martin,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,Aston Martin,British
6,16,7,1:18.218,1:17.390,1:17.369,leclerc,16,LEC,http://en.wikipedia.org/wiki/Charles_Leclerc,Charles,Leclerc,1997-10-16,Monegasque,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian
7,23,8,1:17.962,1:17.761,1:17.609,albon,23,ALB,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander,Albon,1996-03-23,Thai,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British
8,10,9,1:18.312,1:17.574,1:17.675,gasly,10,GAS,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre,Gasly,1996-02-07,French,alpine,http://en.wikipedia.org/wiki/Alpine_F1_Team,Alpine F1 Team,French
9,27,10,1:18.029,1:17.412,1:17.735,hulkenberg,27,HUL,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,Nico,Hülkenberg,1987-08-19,German,haas,http://en.wikipedia.org/wiki/Haas_F1_Team,Haas F1 Team,American
